# CD4_P1_FT

### Variable Paths, Execution Environments

In [ ]:
modelname = 'CD4_P1_FT'

I use 3 Local Machines & 2 Cloud Compute Engines (Google, Kaggle). The appropriate paths for each platform of execution are declared here.

In [ ]:
Kaggle = 0
Colab = 0
Local = 1
LocalRM = 0
LocalOldLaptop = 0

# Kaggle Notebooks
Kaggle_Dataset = '/kaggle/input/catsdogsconv/KaggleCatsDogsConv'
Kaggle_SavedModels = '/kaggle/working/SavedModels'
Kaggle_TrainingHistory = '/kaggle/working/TrainingHistory'

# Google Drive
GD_Dataset = '/content/drive/MyDrive/Datasets/KaggleCatsDogs'
GD_SavedModels = '/content/drive/MyDrive/NotebookWorkspace/SavedModels'
CD_TrainingHistory = '/content/drive/MyDrive/NotebookWorkspace/TrainingHistory'

# Local Directories
Lc_Dataset = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\DatasetConv'
Lc_SavedModels = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\SavedModels'
Lc_TrainingHistory = 'C:\\Programming_Files\\JupyterVSCode\\Binary_Classification_Transfer_Learning\\CatsDogs\\Docs_Reports\\RawTrainingData'

Lc_RM_Dataset = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\Dataset\\KaggleCatsDogsConv"
Lc_RM_SavedModels = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\SavedModels"
Lc_RM_TrainingHistory = "C:\\Users\\arisi\\Documents\\VSCode\\CatsDogs\\Docs_Reports\\RawTrainingData"

Lc_Old_Dataset = 'Test'
Lc_Old_SavedModels = 'Test'
Lc_Old_TrainingHistory = 'Test'

In [ ]:
if Kaggle:
    data_dir = Kaggle_Dataset
    SavedModelsPath = Kaggle_SavedModels
    TrainingHistoryPath = Kaggle_TrainingHistory
if Colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = GD_Dataset
    SavedModelsPath = GD_SavedModels
    TrainingHistoryPath = CD_TrainingHistory
if Local:
    data_dir = Lc_Dataset
    SavedModelsPath = Lc_SavedModels
    TrainingHistoryPath = Lc_TrainingHistory
if LocalRM:
    data_dir = Lc_RM_Dataset
    SavedModelsPath = Lc_RM_SavedModels
    TrainingHistoryPath = Lc_RM_TrainingHistory
if LocalOldLaptop:
    data_dir = Lc_Old_Dataset
    SavedModelsPath = Lc_Old_SavedModels
    TrainingHistoryPath = Lc_Old_TrainingHistory

### CD 2 Model, P1 Parameters

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16


### Load the Dataset

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

In [ ]:
# Load Datasets
train_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='training',     # Use the 'training' subset
    seed=123
)

val_dataset = image_dataset_from_directory(
    data_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary',
    validation_split=0.2,  # 20% for validation
    subset='validation',   # Use the 'validation' subset
    seed=123
)

### Preprocessing & Augmentation

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
# Augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),  # 10% random rotation
    layers.RandomZoom(0.1),      # 10% zoom
    layers.RandomTranslation(0.1, 0.1),  # Random height and width shift
    layers.RandomBrightness(0.2)
])

# Augment the training data
def augment_img(image, label):
    image = data_augmentation(image)  # Apply augmentations
    return image, label

train_dataset = train_dataset.map(augment_img)

In [ ]:
# Apply VGG-16 preprocessing
def preprocess_img(image, label):
    image = preprocess_input(image)  # Apply VGG16-specific preprocessing
    return image, label

train_dataset = train_dataset.map(preprocess_img)
val_dataset = val_dataset.map(preprocess_img)

### Load Model with Trained Head

In [ ]:
filepath = ''
model = tf.keras.models.load_model(filepath)

### Unfreeze Last VGG Block & Verify

In [ ]:
# Unfreeze the last few layers (e.g., last 4 layers)
for layer in model.layers[-4:]:
    layer.trainable = True

# Optionally, print trainable status
for i, layer in enumerate(model.layers):
    print(f"Layer {i}: {layer.name}, Trainable: {layer.trainable}")

In [ ]:
for layer in model.layers[0].layers:
    if layer.name in ['block5_conv1', 'block5_conv2', 'block5_conv3']:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
def print_model_layers(model, indent=0):
    for layer in model.layers:
        print(" " * indent + f"- {layer.name} ({layer.__class__.__name__}), Trainable: {layer.trainable}")
        # If this layer has sublayers (like Functional or Sequential models)
        if hasattr(layer, 'layers'):
            print_model_layers(layer, indent + 2)

print_model_layers(model)

print(model.optimizer.get_config())

### Compile

In [ ]:
name = f'{modelname}'
checkpoint_path = f"{SavedModelsPath}\\{name}_{{epoch:03d}}_val{{val_loss:.4f}}.keras"

from tensorflow.keras.callbacks import ModelCheckpoint

# Create the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_freq='epoch',              # Save every epoch
    save_weights_only=False,
    save_best_only=False,           # Save every time, not just best
    monitor='val_loss',
    verbose=1,
)

from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)

In [ ]:
from tensorflow.keras.optimizers import Adam
optimizer = Adam(learning_rate=0.00001)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

### Train & Save Results

In [ ]:
history = model.fit(
    train_dataset,
    epochs=6,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, lr_scheduler]
)

In [ ]:
import json

name = f'{modelname}'
filepath = f"{TrainingHistoryPath}\\{name}.json"
with open(filepath, 'w') as f:
    json.dump(history.history, f)

### Continue Training (2)

In [ ]:
history2 = model.fit(
    train_dataset,
    epochs=5,
    validation_data=val_dataset,
    callbacks=[checkpoint_callback, lr_scheduler]
)

In [ ]:
import json

name = f'{modelname}_continue'
filepath = f"{TrainingHistoryPath}\\{name}.json"
with open(filepath, 'w') as f:
    json.dump(history2.history, f)